<a href="https://colab.research.google.com/github/DiwakarBasnet/Neural_Network/blob/main/Sales_Prediction/Sales_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Big Mart Sales Prediction
-> Multilayer Perceptron (MLP)

In [1]:
# importing required libraries
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# reading dataset
train_data = 'https://raw.githubusercontent.com/DiwakarBasnet/Neural_Network/main/Sales_Prediction/train_file.csv'
dataset = pd.read_csv(train_data)

##1.Data Preprocessing

In [3]:
dataset.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [4]:
# checking missing values in table
dataset.isnull().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [5]:
dataset.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object

###I. Filling missing values

In [6]:
# filling missing values of continuous data with mean
dataset['Item_Weight'].fillna(dataset['Item_Weight'].mean(), inplace=True)

# filling missing values of categorical data with mode
dataset['Outlet_Size'].fillna(dataset['Outlet_Size'].mode()[0], inplace=True)

In [7]:
# checking missing values after imputation
dataset.isnull().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

###II. Converting categorical data to numbers

In [8]:
print(dataset.Item_Fat_Content.value_counts())
print(dataset.Item_Type.value_counts())
print(dataset.Outlet_Size.value_counts())
print(dataset.Outlet_Location_Type.value_counts())
print(dataset.Outlet_Type.value_counts())

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64
Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64
Medium    5203
Small     2388
High       932
Name: Outlet_Size, dtype: int64
Tier 3    3350
Tier 2    2785
Tier 1    2388
Name: Outlet_Location_Type, dtype: int64
Supermarket Type1    5577
Grocery Store        1083
Supermarket Type3     935
Supermarket Type2     928
Name: Outlet_Type, dtype: int64


In [9]:
dataset['Item_Fat_Content'] = dataset['Item_Fat_Content'].map({'LF':0, 'Low Fat':0, 'low fat':0, 'Regular':1, 'reg':1})
dataset['Item_Type'] = dataset['Item_Type'].map({'Fruits and Vegetables':0, 'Snack Foods':1, 'Household':2, 'Frozen Foods':3, 'Dairy':4,
                                                 'Canned':5, 'Baking Goods':6, 'Health and Hygiene':7, 'Soft Drinks':8, 'Meat':9,
                                                 'Breads':10, 'Hard Drinks':11, 'Others':12, 'Starchy Foods':13, 'Breakfast':14, 'Seafood':15})
dataset['Outlet_Size'] = dataset['Outlet_Size'].map({'Small':0, 'Medium':1, 'High':2})
dataset['Outlet_Location_Type'] = dataset['Outlet_Location_Type'].map({'Tier 1':0, 'Tier 2':1, 'Tier 3':2})
dataset['Outlet_Type'] = dataset['Outlet_Type'].map({'Grocery Store':0, 'Supermarket Type1':1, 'Supermarket Type2':2, 'Supermarket Type3':3})

In [10]:
dataset.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,0,0.016047,4,249.8092,OUT049,1999,1,0,1,3735.1380
1,DRC01,5.92,1,0.019278,8,48.2692,OUT018,2009,1,2,2,443.4228
2,FDN15,17.50,0,0.016760,9,141.6180,OUT049,1999,1,0,1,2097.2700
3,FDX07,19.20,1,0.000000,0,182.0950,OUT010,1998,1,2,0,732.3800
4,NCD19,8.93,0,0.000000,2,53.8614,OUT013,1987,2,2,1,994.7052


###III. Normalization

In [11]:
# bringing the variables in the range of 0 to 1
for i in dataset.columns[1:6]:
  dataset[i] = (dataset[i] - dataset[i].min()) / (dataset[i].max() - dataset[i].min())

for i in dataset.columns[7:]:
  dataset[i] = (dataset[i] - dataset[i].min()) / (dataset[i].max() - dataset[i].min())

In [12]:
dataset.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,0.282525,0.0,0.048866,0.266667,0.927507,OUT049,0.583333,0.5,0.0,0.333333,0.283587
1,DRC01,0.081274,1.0,0.058705,0.533333,0.072068,OUT018,1.000000,0.5,1.0,0.666667,0.031419
2,FDN15,0.770765,0.0,0.051037,0.600000,0.468288,OUT049,0.583333,0.5,0.0,0.333333,0.158115
3,FDX07,0.871986,1.0,0.000000,0.000000,0.640093,OUT010,0.541667,0.5,1.0,0.000000,0.053555
4,NCD19,0.260494,0.0,0.000000,0.133333,0.095805,OUT013,0.083333,1.0,1.0,0.333333,0.073651


In [13]:
dataset.to_csv('normalized_train.csv', index=False)

In [14]:
data = pd.read_csv('normalized_train.csv')
# removing Item_Identifier and Outlet_Identifier
data = data.drop(['Item_Identifier', 'Outlet_Identifier'], axis=1)
data.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,0.282525,0.0,0.048866,0.266667,0.927507,0.583333,0.5,0.0,0.333333,0.283587
1,0.081274,1.0,0.058705,0.533333,0.072068,1.000000,0.5,1.0,0.666667,0.031419
2,0.770765,0.0,0.051037,0.600000,0.468288,0.583333,0.5,0.0,0.333333,0.158115
3,0.871986,1.0,0.000000,0.000000,0.640093,0.541667,0.5,1.0,0.000000,0.053555
4,0.260494,0.0,0.000000,0.133333,0.095805,0.083333,1.0,1.0,0.333333,0.073651


In [15]:
# storing all the features as X
X = data.drop('Item_Outlet_Sales', axis=1)
# storing target as y
y = data['Item_Outlet_Sales']
# shape of X and y
X.shape, y.shape

((8523, 9), (8523,))

In [16]:
##### creating training and validation set #####
# random state to regenerate same train and validation set
# test size 0.2 will keep 20% data in validation and 80% train set

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

(X_train.shape, y_train.shape), (X_test.shape, y_test.shape)

(((6818, 9), (6818,)), ((1705, 9), (1705,)))

##2.Defining the architecture of model

In [17]:
import keras
import tensorflow as tf
from keras import backend as K
from keras.models import Sequential
from keras.layers import InputLayer, Dense

In [18]:
# defining input neurons
input_neurons = X_train.shape[1]
# defining output neurons
output_neurons = 1
# defining hidden layers and neurons in each layer
num_of_hidden_layers = 3
neuron_hidden_layer_1 = 30
neuron_hidden_layer_2 = 15
neuron_hidden_layer_3 = 5

In [19]:
from logging import INFO
model = Sequential()
model.add(InputLayer(input_shape=(input_neurons,)))
model.add(Dense(units=neuron_hidden_layer_1, activation='relu'))
model.add(Dense(units=neuron_hidden_layer_2, activation='relu'))
model.add(Dense(units=neuron_hidden_layer_3, activation='relu'))
model.add(Dense(units=output_neurons, activation='linear'))

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 30)                300       
                                                                 
 dense_1 (Dense)             (None, 15)                465       
                                                                 
 dense_2 (Dense)             (None, 5)                 80        
                                                                 
 dense_3 (Dense)             (None, 1)                 6         
                                                                 
Total params: 851
Trainable params: 851
Non-trainable params: 0
_________________________________________________________________


In [21]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

##### Compiling the model #####
model.compile(loss = root_mean_squared_error, optimizer = 'Adam', metrics = ['mean_squared_error'])

##3.Train model

In [22]:
model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=250)

Epoch 1/250
214/214 [==============================] - 1s 3ms/step - loss: 0.1092 - mean_squared_error: 0.0129 - val_loss: 0.0823 - val_mean_squared_error: 0.0071
Epoch 2/250
214/214 [==============================] - 1s 2ms/step - loss: 0.0868 - mean_squared_error: 0.0078 - val_loss: 0.0796 - val_mean_squared_error: 0.0066
Epoch 3/250
214/214 [==============================] - 1s 2ms/step - loss: 0.0852 - mean_squared_error: 0.0075 - val_loss: 0.0781 - val_mean_squared_error: 0.0064
Epoch 4/250
214/214 [==============================] - 1s 3ms/step - loss: 0.0843 - mean_squared_error: 0.0073 - val_loss: 0.0781 - val_mean_squared_error: 0.0064
Epoch 5/250
214/214 [==============================] - 1s 4ms/step - loss: 0.0840 - mean_squared_error: 0.0073 - val_loss: 0.0791 - val_mean_squared_error: 0.0065
Epoch 6/250
214/214 [==============================] - 1s 3ms/step - loss: 0.0835 - mean_squared_error: 0.0072 - val_loss: 0.0770 - val_mean_squared_error: 0.0062
Epoch 7/250
214/214 [=

In [23]:
from sklearn.metrics import r2_score
##### Evaluating model performance on validation set #####
prediction = model.predict(X_test)

# calculating the accuracy on validation set
r2_score(y_test, prediction)

0.5977613519319616

##4.Test Data Preprocessing

In [24]:
test_data = 'https://raw.githubusercontent.com/DiwakarBasnet/Neural_Network/main/Sales_Prediction/test_file.csv'
check = pd.read_csv(test_data)

In [25]:
check.isnull().sum()

Item_Identifier                 0
Item_Weight                   976
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  1606
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64

###I. Filling Missing Values

In [26]:
# filling missing values of continuous data with mean
check['Item_Weight'].fillna(check['Item_Weight'].mean(), inplace=True)

# filling missing values of categorical data with mode
check['Outlet_Size'].fillna(check['Outlet_Size'].mode()[0], inplace=True)

In [27]:
# checking missing values after imputation
check.isnull().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
dtype: int64

###II. Converting categorical data to numerical

In [28]:
check['Item_Fat_Content'] = check['Item_Fat_Content'].map({'LF':0, 'Low Fat':0, 'low fat':0, 'Regular':1, 'reg':1})
check['Item_Type'] = check['Item_Type'].map({'Fruits and Vegetables':0, 'Snack Foods':1, 'Household':2, 'Frozen Foods':3, 'Dairy':4,
                                                 'Canned':5, 'Baking Goods':6, 'Health and Hygiene':7, 'Soft Drinks':8, 'Meat':9,
                                                 'Breads':10, 'Hard Drinks':11, 'Others':12, 'Starchy Foods':13, 'Breakfast':14, 'Seafood':15})
check['Outlet_Size'] = check['Outlet_Size'].map({'Small':0, 'Medium':1, 'High':2})
check['Outlet_Location_Type'] = check['Outlet_Location_Type'].map({'Tier 1':0, 'Tier 2':1, 'Tier 3':2})
check['Outlet_Type'] = check['Outlet_Type'].map({'Grocery Store':0, 'Supermarket Type1':1, 'Supermarket Type2':2, 'Supermarket Type3':3})

In [29]:
check.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDW58,20.750000,0,0.007565,1,107.8622,OUT049,1999,1,0,1
1,FDW14,8.300000,1,0.038428,4,87.3198,OUT017,2007,1,1,1
2,NCN55,14.600000,0,0.099575,12,241.7538,OUT010,1998,1,2,0
3,FDQ58,7.315000,0,0.015388,1,155.0340,OUT017,2007,1,1,1
4,FDY38,12.695633,1,0.118599,4,234.2300,OUT027,1985,1,2,3


###III. Normalization

In [30]:
# bringing the variables in the range of 0 to 1
for i in check.columns[1:6]:
  check[i] = (check[i] - check[i].min()) / (check[i].max() - check[i].min())

for i in check.columns[7:]:
  check[i] = (check[i] - check[i].min()) / (check[i].max() - check[i].min())

In [31]:
check.to_csv('normalized_test.csv', index=False)

In [32]:
data_test = pd.read_csv('normalized_test.csv')

In [33]:
# removing Item_Identifier and Outlet_Identifier
new_test_data = data_test.drop(['Item_Identifier', 'Outlet_Identifier'], axis=1)

In [34]:
new_test_data.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,0.964275,0.0,0.023374,0.066667,0.323413,0.583333,0.5,0.0,0.333333
1,0.222983,1.0,0.118737,0.266667,0.235849,0.916667,0.5,0.5,0.333333
2,0.598095,0.0,0.307674,0.800000,0.894140,0.541667,0.5,1.0,0.000000
3,0.164335,0.0,0.047548,0.066667,0.524488,0.916667,0.5,0.5,0.333333
4,0.484706,1.0,0.366458,0.266667,0.862069,0.000000,0.5,1.0,1.000000


In [35]:
pred = model.predict(new_test_data)

In [39]:
df = pd.DataFrame()
df['Item_Identifier'] = data_test['Item_Identifier']
df['Outlet_Identifier'] = data_test['Outlet_Identifier']
df['Item_Outlet_Sales']= pred

df

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,0.111780
1,FDW14,OUT017,0.096146
2,NCN55,OUT010,0.040060
3,FDQ58,OUT017,0.225286
4,FDY38,OUT027,0.463918
...,...,...,...
5676,FDB58,OUT046,0.169242
5677,FDD47,OUT018,0.201875
5678,NCO17,OUT045,0.155101
5679,FDJ26,OUT017,0.363713


In [40]:
df.to_csv('solution.csv', index=False)